In [1]:
import pandas as pd
import numpy as np
import geocoder

path_to_dataset = '/Users/mihaithpc/Desktop/Europe_Datathon/BigSupplyCo_Data_Files /'

customers = pd.read_csv(path_to_dataset + 'BigSupplyCo_Customers.csv')
orders = pd.read_csv(path_to_dataset + 'BigSupplyCo_Orders.csv')
products = pd.read_csv(path_to_dataset + 'BigSupplyCo_Products.csv')
departments = pd.read_csv(path_to_dataset + 'BigSupplyCo_Departments_Cleaned.csv')
categories = pd.read_csv(path_to_dataset + 'BigSupplyCo_Categories.csv')

customers['Customer Full Name'] = customers['Customer Fname'].astype(str)+ \
                                  ' ' + customers['Customer Lname'].astype(str)
customers['Customer Zipcode'] = customers['Customer Zipcode'].fillna(0)
customers = customers.drop(['Customer Email', 
                            'Customer Password',
                            'Customer Fname',
                            'Customer Lname',
                            'Customer Street'], axis=1)
                            #'Customer Zipcode'], axis=1) shall we drop this?
orders = orders.drop(['Order Zipcode',  # too few non-null values
                      'Order Item Discount',
                      'Order Item Discount Rate',
                      'Sales',  # this and two above determine Order Item Total - after discounts
                      'Order Item Id',  # does not join with any other column in a different table
                      'Order Item Quantity',  # I think the quantity is not important: perfection should occur for every order
                      'Unnamed: 0',
                      'Order Id'], axis=1) 
products = products.drop(['Product Description',  # NaN everywhere, hence remove it
                          'Product Image',  # useless, hence remove it 
                          'Product Status',  # only one value for this column, hence remove it
                          'Product Name'], axis=1)  # group by categories rather than by each of the same category

def get_country(row):
    return geocoder.osm([row['Latitude'], row['Longitude']], method='reverse').json['country']
departments = departments.drop(['Latitude',
                               'Longitude'], axis=1)

join_dept = pd.merge(orders, departments, left_on='Order Department Id', right_on='Department Id').drop(
            columns=['Department Id'])
join_cust = pd.merge(join_dept, customers, left_on='Order Customer Id', right_on='Customer Id').drop(
            columns=['Customer Id'])
join_prod = pd.merge(join_cust, products, left_on='Order Item Cardprod Id', right_on='Product Card Id').drop(
            columns=['Product Card Id'])
data_ready = pd.merge(join_prod, categories, left_on='Product Category Id', right_on='Category Id').drop(
            columns=['Category Id'])

data_ready['Date'] = pd.to_datetime(data_ready['order date (DateOrders)'],
                                    infer_datetime_format=True).dt.date
data_ready.drop(columns=['order date (DateOrders)'], inplace=True)

data_ready['Date'] = pd.to_datetime(data_ready['Date'], infer_datetime_format=True).astype('datetime64[ns]')
data_ready.set_index('Date', inplace=True)
data_ready.sort_index(inplace=True)

